<a href="https://colab.research.google.com/github/yangsu0/class_machine_learning/blob/master/ML_Assignment11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

review_data = load_files(r"movie_review")
X, y = review_data.data, review_data.target

documents = []

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False) #x = vector, y=label


#(1401, 1500), (601, 1500), (1401,), (601,)
y_train = y_train.reshape((-1, 1))
y_test = y_test.reshape((-1,1))
X_train = X_train.T
X_test = X_test.T
y_train = y_train.T
y_test = y_test.T

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
#weight & bias parameters

u=np.random.randn(60,1500)
v=np.random.randn(1,60)

b1 = np.zeros((60, 1))
b2 = np.zeros((1, 1))

lamb = 0.725

In [0]:
#define functions
def sigmoid(z):
  return 1/(1+np.exp(-z))

def sigmoid_prime(z):
  return sigmoid(z) * (1 - sigmoid(z))

def object_func(label, sig):
  n = len(u)*len(u[0]) + len(v)*len(v[0])
  return -np.mean(label*np.log(sig) + (1-label)*np.log(1-sig)) + (lamb/(2*n))*(np.sum(np.square(u)) + np.sum(np.square(v)))

In [0]:
loss_train_list = []
loss_test_list = []
acc_train_list = []
acc_test_list = []
u_list = []
v_list = []

b1_list = []
b2_list = []


In [0]:
#y' = u*x
#y = sigmoid(y')
#z' = v*y
#z = sigmoid(z')


def gradient(z_,y_,z,y):
  n = len(u)*len(u[0]) + len(v)*len(v[0]) 
  dz = (z-y_train)/z*(1-z)
  dz_ = np.multiply(dz, sigmoid_prime(z_))
  dv = (1/1401)* np.dot(dz_,z.T)+(1/n)*(lamb)*v
  db2 = (1/1401)* np.sum(dz_, axis=1, keepdims=True)
  dy = np.dot(v.T,dz_)
  dy_ = np.multiply(dy, sigmoid_prime(y_))
  du = (1/1401)* np.dot(dy_,X_train.T)+(1/n)*(lamb)*u
  db1 = (1/1401)* np.sum(dy_, axis=1, keepdims=True)

  return du, dv, db1, db2

In [0]:
def accuracy(label, sig):
  data_len = label.shape[1]
  cnt = 0

  for i in range(data_len):
    if sig[:,i] < 0.5:
      l = 0
    elif sig[:,i] >= 0.5:
      l = 1
    if label[:,i] == l:
      cnt += 1
  return (cnt/data_len) * 100